In [ ]:
import os
import re
import glob
import pandas as pd
import matplotlib.pyplot as plt

print(os.getcwd())

def get_info(file):
    prediction_rate=1   # for perfect condition
    speedup = 0
    cpi = 0
    lines = open(file).readlines()
    for line in lines:
        if line.find('bpred_dir_rate') > 0:
            prediction_rate = float(re.findall(r'0+\.\d+', line)[0])
            # print(prediction_rate)
        if line.find('sim_CPI') == 0:
            cpi = float(re.findall(r'\d+\.\d+', line)[0])
            # print(cpi)
    return prediction_rate,cpi

app_name_list=[
    "dijkstra_small",
    # "stringsearch-cabce", 
    # "qsort_small", 
    # "gsm-untoast", 
    # "jpeg-cjpeg"
    ]


IF_Queue_size_list=[2,4,8]
RUU_size_list = [16,32,64,128]
ALUs_list = [1,2,3,4,5,6,7,8]
Mults_list = [1,2,3,4]
Issue_wrong_path_list = ["True", "False"]


flist = glob.glob("./simhome/configs/task_2_2/ifqs_2_ruus_16_lsq_8_alus_*_mults_*_iwp_False")
flist.sort(reverse=True)
print(len(flist))
# csv=open("task_2_2.csv",'w')
app_perf_map={}
app_perf_cpi_map={}
# print("app,"+",".join([p.split("/")[-1] for p in flist]), file=csv)

app = "jpeg-cjpeg"
    # "dijkstra_small"
    # "stringsearch-cabce", 
    # "qsort_small", 
    # "gsm-untoast", 
    # "jpeg-cjpeg"
## to show the alus effection.
def show_cpu(ifqs,ruus,lsq,mults,iwp):
    cpi_list=[]
    for alu in ALUs_list:
        item = "./simhome/configs/task_2_2/ifqs_%d_ruus_%d_lsq_%d_alus_%s_mults_%d_iwp_%s" \
            % (ifqs,ruus,lsq,alu,mults,iwp)
        stat_file_path = os.path.join(item, "Stats_"+ app + ".txt")
        rate,cpi = get_info(stat_file_path)
        # print(rate, end=',', file=csv)
        cpi_list.append(cpi)
    print(cpi_list)
    return cpi_list

plt.figure(figsize=(12, 4))
plt.title(app)

plt.subplot(141)
print("ALUs vs IFQS")
plt.ticklabel_format(useOffset=False, style='plain')
plt.title("ALUs vs IFQ,\n RUU=16,\nLSQ=8,\n Mults=1,\n WRONG_PATH=False")
plt.plot([i for i in range(len(ALUs_list))], show_cpu(2,16,8,1,"False"), label="IFQ=2")
plt.plot([i for i in range(len(ALUs_list))], show_cpu(4,16,8,1,"False"), label="IFQ=4")
plt.plot([i for i in range(len(ALUs_list))], show_cpu(8,16,8,1,"False"), label="IFQ=8")
plt.legend()
plt.xlabel("ALUs number")
plt.ylabel("CPI")

# we can notice that as IFQS increas CPI reduces sligthly -> IFQS = 8
plt.subplot(142)
print("ALUs vs RUUs (LSQ= RUUs/2), IFQS=8")
plt.title("ALUs vs RUUs,\n LSQ = RUUs/2,\n IFQS=8 \n WRONG_PATH=False")
plt.plot([i for i in range(len(ALUs_list))], show_cpu(8,16,8,1,"False"), label="RUU=16")
plt.plot([i for i in range(len(ALUs_list))], show_cpu(8,32,16,1,"False"), label="RUU=32")
plt.plot([i for i in range(len(ALUs_list))], show_cpu(8,64,32,1,"False"), label="RUU=64")
plt.plot([i for i in range(len(ALUs_list))], show_cpu(8,128,64,1,"False"), label="RUU=128")
plt.legend()
plt.xlabel("ALUs number")
plt.ylabel("CPI")

# we can notic that as RUUs increas CPI reduces -> RUUs = 128, LSQ =64
plt.subplot(143)
print("ALUs vs Mults, IFQS=8, RUUs=128, LSQ=64 ")
plt.title("ALUs vs Mults, \n IFQS=8, \n RUUs=128, \n LSQ=64 \n WRONG_PATH=False")
plt.plot([i for i in range(len(ALUs_list))], show_cpu(8,128,64,1,"False"), label="Mults=1")
plt.plot([i for i in range(len(ALUs_list))], show_cpu(8,128,64,2,"False"), label="Mults=2")
plt.plot([i for i in range(len(ALUs_list))], show_cpu(8,128,64,3,"False"), label="Mults=3")
plt.plot([i for i in range(len(ALUs_list))], show_cpu(8,128,64,4,"False"), label="Mults=4")
plt.legend()
plt.xlabel("ALUs number")
plt.ylabel("CPI")

# not bad effect on this app, so we chose MULTs = 4
plt.subplot(144)
print("ALUs vs ISSUE_WRONG_PATH(True or False), IFQS=8, RUUs=128, LSQ=64, Mults=4 ")
plt.title("ALUs vs WRONG_PATH,\n IFQS=8,\n RUUs=128,\n LSQ=64, \n Mults=4 ")
plt.plot([i for i in range(len(ALUs_list))], show_cpu(8,128,64,4,"True"), label="True")
plt.plot([i for i in range(len(ALUs_list))], show_cpu(8,128,64,4,"False"), label="False")
plt.legend()
plt.xlabel("ALUs number")
plt.ylabel("CPI")

plt.tight_layout() 
plt.show()


